In [94]:
%matplotlib inline

import numpy as np
import matplotlib.pylab as plt

# not sure I'll need all this, grabbing it all just in case

import astropy.io.fits as fits
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy.coordinates import get_sun, get_body, get_moon

import pytz

from astroplan import Observer, FixedTarget, time_grid_from_range, observability_table, moon_illumination
from astroplan import AirmassConstraint, MoonSeparationConstraint

In [95]:
apo = Observer.at_site('apo')

In [96]:
reference_time = Time('2018-5-18')

In [97]:
tz_1 = pytz.timezone("America/Denver")

In [98]:
constraints = [MoonSeparationConstraint(25*u.deg)]
constraints.append(AirmassConstraint(2))

In [99]:
astro_set1 = apo.twilight_evening_astronomical(reference_time, which='nearest')
astro_rise1 = apo.twilight_morning_astronomical(reference_time, which='next')
observing_length1 = (astro_rise1 - astro_set1).to(u.h)

print("Astronomical Evening Twilight starts at {0.iso} UTC".format(astro_set1))
print("Astronomical Morning Twilight starts at {0.iso} UTC".format(astro_rise1))
print("You can observe for {0:.1f} during the observing window.".format(observing_length1))

Astronomical Evening Twilight starts at 2018-05-18 03:32:26.384 UTC
Astronomical Morning Twilight starts at 2018-05-18 10:26:39.610 UTC
You can observe for 6.9 h during the observing window.


In [100]:
print(apo.moon_phase(reference_time))

2.5390025074076488 rad


In [101]:
# right on

In [102]:
apo.moon_illumination(reference_time)

0.08806482926481685

In [103]:
# much better than last time we tried this

In [104]:
obs_range = [astro_set1, astro_rise1]

# Here is where the target list of galaxies may be found:

In [105]:
# Some of these are probably too faint (I couldn't actually remember what you said our max magnitude was likely to be,
# so I just used 20 as a cutoff), but I figured a larger list of targets couldn't hurt. I was also just selecting these
# by perusing SDSS images and picking galxies when they had low enough magnitudes, which means a lot of them were
# overexposed. So it may not quite have worked out like I was hoping morphology-wise, but best laid plans and such...

target_array = [['SDSS J150051+031511.5', 225.212685, 3.253196], 
               ['SDSS J150108.2+032030.1', 225.284286, 3.341717],
               ['SDSS J150052.2+032729.9', 225.21773, 3.458323],
               ['SDSS J150133.1+033548.9', 225.388195, 3.596924],
               ['SDSS J150157.3+034842', 225.488932, 3.811684],
               ['SDSS J150409.8+034314.2', 226.041027, 3.720614],
               ['SDSS J150400.7+035504.8', 226.003004, 3.918004],
               ['SDSS J150351.9+035441.6', 225.966388, 3.911572],
               ['SDSS J150359.4+041648.6', 225.997552, 4.280173],
               ['SDSS J150337.1+042017.6', 225.904716, 4.338227],
               ['SDSS J154401.8+023055.2', 236.007775, 2.515347],
               ['SDSS J154346.4+022624.2', 235.943454, 2.440082],
               ['SDSS J154430.2+032212.2', 236.125887, 3.370082],
               ['SDSS J154233.6+032937.2', 235.640167, 3.493672],
               ['SDSS J153901+032803.8', 234.754254, 3.467724],
               ['SDSS J153318+031922.5', 233.325088, 3.32294],
               ['SDSS J151211.3+041920.4', 228.047258, 4.322337],
               ['SDSS J150928+035156.8', 227.366834, 3.865784],
               ['SDSS J150152.3+032137.6', 225.468068, 3.360446],
               ['SDSS J150012.9+031541.7', 225.054138, 3.261601],
               ['SDSS J150101.4+025351.1', 225.256033, 2.897534],
               ['SDSS J150111.2+014207.1', 225.296805, 1.701983],
               ['SDSS J150000.4+015328.7', 225.001667, 1.891325],
               ['SDSS J144455.9+015718', 221.232992, 1.955003],
               ['SDSS J144725.6+020208.6', 221.856789, 2.035734],
               ['SDSS J144611.1-001322.8', 221.54635, -0.223017]]

In [106]:
targets = [FixedTarget(coord=SkyCoord(ra = RA*u.deg, dec = DEC*u.deg), name=Name)
           for Name, RA, DEC in target_array]

In [107]:
observing_table = observability_table(constraints, apo, targets, time_range=obs_range)

print(observing_table)

      target name       ever observable ... fraction of time observable
----------------------- --------------- ... ---------------------------
  SDSS J150051+031511.5            True ...                         1.0
SDSS J150108.2+032030.1            True ...                         1.0
SDSS J150052.2+032729.9            True ...                         1.0
SDSS J150133.1+033548.9            True ...                         1.0
  SDSS J150157.3+034842            True ...                         1.0
SDSS J150409.8+034314.2            True ...                         1.0
SDSS J150400.7+035504.8            True ...                         1.0
SDSS J150351.9+035441.6            True ...                         1.0
SDSS J150359.4+041648.6            True ...                         1.0
SDSS J150337.1+042017.6            True ...                         1.0
                    ...             ... ...                         ...
  SDSS J153318+031922.5            True ...                     

In [108]:
for i, target_table in enumerate(targets):

    if observing_table['ever observable'][i]:
        name = observing_table['target name'][i]
        obj_frac = observing_table['fraction of time observable'][i]
        obj_time = obj_frac * observing_length1
        output = "You can observe {0:s} for {1:.2f}".format(name, obj_time.to(u.h))
        print(output)

You can observe SDSS J150051+031511.5 for 6.90 h
You can observe SDSS J150108.2+032030.1 for 6.90 h
You can observe SDSS J150052.2+032729.9 for 6.90 h
You can observe SDSS J150133.1+033548.9 for 6.90 h
You can observe SDSS J150157.3+034842 for 6.90 h
You can observe SDSS J150409.8+034314.2 for 6.90 h
You can observe SDSS J150400.7+035504.8 for 6.90 h
You can observe SDSS J150351.9+035441.6 for 6.90 h
You can observe SDSS J150359.4+041648.6 for 6.90 h
You can observe SDSS J150337.1+042017.6 for 6.90 h
You can observe SDSS J154401.8+023055.2 for 6.90 h
You can observe SDSS J154346.4+022624.2 for 6.90 h
You can observe SDSS J154430.2+032212.2 for 6.90 h
You can observe SDSS J154233.6+032937.2 for 6.90 h
You can observe SDSS J153901+032803.8 for 6.90 h
You can observe SDSS J153318+031922.5 for 6.90 h
You can observe SDSS J151211.3+041920.4 for 6.90 h
You can observe SDSS J150928+035156.8 for 6.90 h
You can observe SDSS J150152.3+032137.6 for 6.90 h
You can observe SDSS J150012.9+031541.7 f

In [109]:
for target in targets:
    if observing_table['ever observable'][i]:
        
        rise_time = apo.target_rise_time(reference_time, target, which='nearest')
    
        #set_time = apo.target_set_time(target)
    
        # flux it
        spluh = rise_time.to_datetime(tz_1)
    
        print(spluh)

2018-05-17 18:14:17.741717-06:00
2018-05-17 18:14:21.137554-06:00
2018-05-17 18:13:47.139629-06:00
2018-05-17 18:14:06.397179-06:00
2018-05-17 18:13:57.152168-06:00
2018-05-17 18:16:23.327074-06:00
2018-05-17 18:15:43.575127-06:00
2018-05-17 18:15:35.816650-06:00
2018-05-17 18:14:45.984291-06:00
2018-05-17 18:14:14.758430-06:00
2018-05-17 18:59:13.468877-06:00
2018-05-17 18:59:09.744013-06:00
2018-05-17 18:57:29.269161-06:00
2018-05-17 18:55:13.882045-06:00
2018-05-17 18:51:46.077057-06:00
2018-05-17 18:46:26.836061-06:00
2018-05-17 18:22:49.580743-06:00
2018-05-17 18:21:17.862673-06:00
2018-05-17 18:15:02.181508-06:00
2018-05-17 18:13:38.528853-06:00
2018-05-17 18:15:23.230518-06:00
2018-05-17 18:18:38.042055-06:00
2018-05-17 18:16:58.154646-06:00
2018-05-16 18:05:42.917271-06:00
2018-05-17 18:04:03.712176-06:00
2018-05-17 18:08:38.468602-06:00


In [110]:
for target in targets:
        
    if observing_table['ever observable'][i]:
        
        set_time = apo.target_set_time(reference_time, target, which='nearest')
    
        #set_time = apo.target_set_time(target)
    
        # flux it
        spleh = set_time.to_datetime(tz_1)
    
        print(spleh)

2018-05-17 06:32:38.090204-06:00
2018-05-17 06:33:08.893457-06:00
2018-05-17 06:33:10.962003-06:00
2018-05-17 06:34:13.202784-06:00
2018-05-17 06:35:10.575195-06:00
2018-05-17 06:37:08.788555-06:00
2018-05-17 06:37:30.213222-06:00
2018-05-17 06:37:20.445703-06:00
2018-05-17 06:38:24.970439-06:00
2018-05-17 06:38:11.725007-06:00
2018-05-17 07:13:50.200030-06:00
2018-05-17 07:13:23.178088-06:00
2018-05-17 07:16:30.409340-06:00
2018-05-17 07:14:53.116486-06:00
2018-05-17 07:11:16.835985-06:00
2018-05-17 07:05:12.157457-06:00
2018-05-17 06:46:42.453437-06:00
2018-05-17 06:42:48.719360-06:00
2018-05-17 06:33:55.785900-06:00
2018-05-17 06:32:01.430093-06:00
2018-05-17 06:31:53.593765-06:00
2018-05-17 06:28:59.103261-06:00
2018-05-17 06:28:17.572446-06:00
2018-05-17 06:13:24.632691-06:00
2018-05-17 06:16:06.504474-06:00
2018-05-17 06:09:04.446453-06:00
